# Substrait as a Query Engine for Apache Iceberg [Experimental]

The abstraction provided by Substrait can be used to execute your query in many query engines. There are two aspects to Substrait. One is a as producer it produces Substrait plans from SQL queries based on data accessible for a query engine. Duckdb and Ibis can be considered as Substrait producers. The other one is consuming Substrait plans. DuckDb can be considered as one such prominent Substrait consumers. 

## Why Schema Evolution support via Substrait?

Supporting schema evolution not a very straightforward task. Apache Iceberg is one such framework which provides this capability to the user by supporting a set of existing query engines. 

If your query engine is not one of them, you will have to implement this from scratch. To breach this gap, we propose a Substrait-based solution which would do the ground work to execute queries in your engine with schema evolution support. 

### Limitations:

1. Your query engine must support Substrait.
2. Python-based solution

We have developed a prototype to enable Schema evolution support via Substrait. Once this module is used, your query engine can run queries enabling schema evolution. 

**Warning**

This module is experimental.

## Demo Objectives

Demonstrate Schema Evolution scenarios using the experimental module we have developed and use Apache Spark to validate the results. 

## Why Apache Spark?

Apache Iceberg supports a few query engines, Apache Spark is one of those query engines. 
We will be using Apache Spark as the go to engine to write the dataset in Apache Iceberg format. Furthermore, we will use Apache Spark to validate the results generated from our module.

## Duckdb as a Query Engine

### Installing Experimental Module

We are installing the `icetrait` Python library from a experimental branch.

In [98]:
!pip install git+https://github.com/vibhatha/pyiceberg_substrait@feat-schema-evolution-s1#egg=icetrait

  Cloning https://github.com/vibhatha/pyiceberg_substrait (to revision feat-schema-evolution-s1) to /tmp/pip-install-wucc5doe/icetrait_e0aaa0715eda4e7080a3a253b407ba36
  Running command git clone --filter=blob:none --quiet https://github.com/vibhatha/pyiceberg_substrait /tmp/pip-install-wucc5doe/icetrait_e0aaa0715eda4e7080a3a253b407ba36
  Running command git checkout -b feat-schema-evolution-s1 --track origin/feat-schema-evolution-s1
  Switched to a new branch 'feat-schema-evolution-s1'
  Branch 'feat-schema-evolution-s1' set up to track remote branch 'feat-schema-evolution-s1' from 'origin'.
  Resolved https://github.com/vibhatha/pyiceberg_substrait to commit 8e61be62f1b70aa74b748487fa743ac652cfaaa9
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for icetrait: filename=icetrait-0.1

**Disable logging**

In [1]:
import os
os.environ['ICETRAIT_LOG_DIR'] = '/home/iceberg/notebooks/icetrait_logs'
os.getenv("ICETRAIT_LOG_DIR")
os.environ['ICETRAIT_LOGGING'] = 'DISABLE'

### Warning

Make sure to **restart the kernel** after installation

In [2]:
from icetrait.substrait.visitor import SubstraitPlanEditor, visit_and_update, RelVisitor, RelUpdateVisitor
from icetrait.duckdb.wrapper import DuckdbSubstrait
import duckdb

## Initialize Spark Environment

We will be using Apache Spark as the tool to save data in Apache Iceberg Format. Also we will be using Apache Spark to validate our results. Please note that Apache Iceberg supports Apache Spark. 

In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.master("local")
    .appName("IcebergPySpark")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.demo.catalog-impl", "org.apache.iceberg.rest.RESTCatalog")
    .config("spark.sql.catalog.demo.uri", "http://rest:8181")
    .config("spark.sql.catalog.demo.s3.endpoint", "http://minio:9000")
    .config("spark.sql.defaultCatalog", "demo")
    .config("spark.eventLog.enabled", "true")
    .config("spark.eventLog.dir", "/home/iceberg/spark-events")
    .config("spark.history.fs.logDirectory", "/home/iceberg/spark-events")
    .config("spark.sql.catalogImplementation", "/home/iceberg/spark-events")
    .getOrCreate()
)
spark

23/05/27 02:09:12 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### Writing Data in Apache Iceberg Format

First, let’s write some data in Apache Iceberg format using Apache Spark. We will be using NYC Taxi driver dataset. 

Drop if there is an existing table and start from scratch. 

In [4]:
%%sql

DROP TABLE IF EXISTS nyc_demo.taxis_sample;

23/05/27 02:09:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [5]:
%%sql

CREATE DATABASE IF NOT EXISTS nyc_demo;

23/05/27 02:09:17 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [6]:
%%sql

show databases;

23/05/27 02:09:17 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


namespace
nyc_demo


In [7]:
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2021-04.parquet")
df_small = df.limit(500)
df_small = df.select(['VendorID',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'airport_fee'])
df_small.write.saveAsTable("nyc_demo.taxis_sample")

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


## Simulating a Simple Query

In [8]:
spark_s_query = "SELECT * FROM nyc_demo.taxis_sample;"
df_sql = spark.sql(spark_s_query)
spark_s_df = df_sql.toPandas()

In [9]:
spark_s_df.head()

,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,1.0,8.40,1.0,N,79,116,1,25.5,3.0,0.5,5.85,0.0,0.3,35.15,2.5,0.0
1,1,1.0,0.90,1.0,N,75,236,2,5.0,3.0,0.5,0.00,0.0,0.3,8.80,2.5,0.0
2,1,1.0,3.40,1.0,N,236,168,2,11.5,3.0,0.5,0.00,0.0,0.3,15.30,2.5,0.0
3,1,1.0,0.00,1.0,N,47,61,1,44.2,0.0,0.5,0.00,0.0,0.3,45.00,0.0,0.0
4,2,1.0,1.96,1.0,N,238,152,1,9.0,0.5,0.5,3.09,0.0,0.3,13.39,0.0,0.0


In [10]:
"""
We expect the user to pass the evolved schema.
"""
## TODO: I think we can probably keep a single connection and use the update query to Spark applied on
## the duckdb connector and update the table.
def setup_duckdb():
    con = duckdb.connect()
    create_schema = "CREATE SCHEMA nyc_demo;"
    creation_query = """
    CREATE TABLE nyc_demo.taxis_sample (
        VendorID              bigint,
        passenger_count       double,
        trip_distance         double,
        RatecodeID            double,
        store_and_fwd_flag    string,
        PULocationID          bigint,
        DOLocationID          bigint,
        payment_type          bigint,
        fare_amount           double,
        extra                 double,
        mta_tax               double,
        tip_amount            double,
        tolls_amount          double,
        improvement_surcharge double,
        total_amount          double,
        congestion_surcharge  double,
        airport_fee           double
    );
    """
    con.execute(create_schema)
    con.execute(creation_query)
    return con

In [11]:
q1 = "SELECT * FROM nyc_demo.taxis_sample;"

In [12]:
!mkdir /home/iceberg/notebooks/s3

mkdir: cannot create directory ‘/home/iceberg/notebooks/s3’: File exists


In [13]:
wrapper = DuckdbSubstrait("default", "/home/iceberg/notebooks/s3", "nyc_demo", q1, setup_duckdb)
wrapper.update_named_table_with_schema()
wrapper.update_with_local_file_paths()

In [14]:
duckdb_res = wrapper.execute()

In [15]:
df_duckdb_substrait = duckdb_res.to_df()
df_duckdb_substrait.head()

,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,1.0,8.40,1.0,N,79,116,1,25.5,3.0,0.5,5.85,0.0,0.3,35.15,2.5,0.0
1,1,1.0,0.90,1.0,N,75,236,2,5.0,3.0,0.5,0.00,0.0,0.3,8.80,2.5,0.0
2,1,1.0,3.40,1.0,N,236,168,2,11.5,3.0,0.5,0.00,0.0,0.3,15.30,2.5,0.0
3,1,1.0,0.00,1.0,N,47,61,1,44.2,0.0,0.5,0.00,0.0,0.3,45.00,0.0,0.0
4,2,1.0,1.96,1.0,N,238,152,1,9.0,0.5,0.5,3.09,0.0,0.3,13.39,0.0,0.0


In [17]:
assert spark_s_df.equals(df_duckdb_substrait)

## Simulating Schema Evolution for Rename Operation

We **rename** the field **`fare_amount`** to **`fare`**. 

In [18]:
%%sql

ALTER TABLE nyc_demo.taxis_sample RENAME COLUMN fare_amount TO fare

23/05/27 02:10:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
23/05/27 02:10:07 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


++
||
++
++

**Since we use Duckdb as the query engine we will have to provide a schema to Duckdb to generate a Substrait plan. This schema is the evolved schema**. 



In [19]:
# instead of `fare_amount` we use `fare` since it is the renamed field. 
def setup_duckdb_for_rename():
    con = duckdb.connect()
    create_schema = "CREATE SCHEMA nyc_demo;"
    creation_query = """
    CREATE TABLE nyc_demo.taxis_sample (
        VendorID              bigint,
        passenger_count       double,
        trip_distance         double,
        RatecodeID            double,
        store_and_fwd_flag    string,
        PULocationID          bigint,
        DOLocationID          bigint,
        payment_type          bigint,
        fare                  double,
        extra                 double,
        mta_tax               double,
        tip_amount            double,
        tolls_amount          double,
        improvement_surcharge double,
        total_amount          double,
        congestion_surcharge  double,
        airport_fee           double
    );
    """
    con.execute(create_schema)
    con.execute(creation_query)
    return con

In [20]:
query_rename = "SELECT fare FROM nyc_demo.taxis_sample;"

In [22]:
wrapper = DuckdbSubstrait("default", "/home/iceberg/notebooks/s3", "nyc_demo", query_rename, setup_duckdb_for_rename)
wrapper.update_named_table_with_schema()
wrapper.update_with_local_file_paths()

In [23]:
duckdb_rename_res = wrapper.execute()

In [24]:
df_duckdb_substrait_rename = duckdb_rename_res.to_df()
df_duckdb_substrait_rename.head()

,fare
0,25.5
1,5.0
2,11.5
3,44.2
4,9.0


In [25]:
df_sql = spark.sql(query_rename)
spark_rename_df = df_sql.toPandas()

In [34]:
assert spark_rename_df.equals(df_duckdb_substrait_rename)

## Simulating Schema Evolution for Add operation

We will make an alteration to the Table by adding a new column called **`fare_per_distance_unit`**.

In [ ]:
%%sql

ALTER TABLE nyc_demo.taxis_sample
ADD COLUMN fare_per_distance_unit float AFTER trip_distance

Then let’s update values for this column. We will do the following operation to add values from existing columns. 

 **`fare_per_distance_unit`** = **`fare`** **/** **`trip_distance`**

In [49]:
%%sql

UPDATE nyc_demo.taxis_sample
SET fare_per_distance_unit = fare/trip_distance

23/05/27 02:23:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

As same as before, we will update the Duckdb schema to the evolved schema. Note that, since the update occurred, column renames are persisted. 

In [50]:
# adding field `fare_per_distance_unit`
def setup_duckdb_for_update():
    con = duckdb.connect()
    create_schema = "CREATE SCHEMA nyc_demo;"
    creation_query = """
    CREATE TABLE nyc_demo.taxis_sample (
        VendorID               bigint,
        passenger_count        double,
        trip_distance          double,
        fare_per_distance_unit float, 
        RatecodeID             double,
        store_and_fwd_flag     string,
        PULocationID           bigint,
        DOLocationID           bigint,
        payment_type           bigint,
        fare                   double,
        extra                  double,
        mta_tax                double,
        tip_amount             double,
        tolls_amount           double,
        improvement_surcharge  double,
        total_amount           double,
        congestion_surcharge   double,
        airport_fee            double
    );
    """
    con.execute(create_schema)
    con.execute(creation_query)
    return con

In [51]:
query_add = "SELECT fare_per_distance_unit, fare FROM nyc_demo.taxis_sample;"

In [52]:
wrapper = DuckdbSubstrait("default", "/home/iceberg/notebooks/s3", "nyc_demo", query_add, setup_duckdb_for_update)
wrapper.update_named_table_with_schema()
wrapper.update_with_local_file_paths()

In [53]:
duckdb_add_res = wrapper.execute()

In [54]:
df_duckdb_substrait_add = duckdb_add_res.to_df()
df_duckdb_substrait_add.head()

,fare_per_distance_unit,fare
0,3.035714,25.5
1,5.555555,5.0
2,3.382353,11.5
3,NaN,44.2
4,4.591837,9.0


In [35]:
df_sql = spark.sql(query_add)
spark_add_df = df_sql.toPandas()
spark_add_df.head()

,fare_per_distance_unit,fare
0,3.035714,25.5
1,5.555555,5.0
2,3.382353,11.5
3,NaN,44.2
4,4.591837,9.0


In [36]:
assert spark_add_df.equals(df_duckdb_substrait_add)

## Simulating Schema Evolution for Drop Operation

Next, we will take a look at a column drop situation. Let’s drop the column **`improvement_surcharge`**. Also next update the Duckdb schema to the evolved schema so that it generates the evolved schema. 

In [37]:
%%sql
ALTER TABLE nyc_demo.taxis_sample
DROP COLUMN improvement_surcharge;

23/05/27 02:15:29 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
23/05/27 02:15:29 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


++
||
++
++

In [55]:
# field `improvement_surcharge` is included duckdb assumes the fullschema to be available and
# it handles selection via projection
def setup_duckdb_for_drop():
    con = duckdb.connect()
    create_schema = "CREATE SCHEMA nyc_demo;"
    creation_query = """
    CREATE TABLE nyc_demo.taxis_sample (
        VendorID               bigint,
        passenger_count        double,
        trip_distance          double,
        fare_per_distance_unit float, 
        RatecodeID             double,
        store_and_fwd_flag     string,
        PULocationID           bigint,
        DOLocationID           bigint,
        payment_type           bigint,
        fare                   double,
        extra                  double,
        mta_tax                double,
        tip_amount             double,
        tolls_amount           double,
        total_amount           double,
        congestion_surcharge   double,
        airport_fee            double
    );
    """
    con.execute(create_schema)
    con.execute(creation_query)
    return con

In [41]:
query_drop = "SELECT * FROM nyc_demo.taxis_sample;"

In [42]:
wrapper = DuckdbSubstrait("default", "/home/iceberg/notebooks/s3", "nyc_demo", query_drop, setup_duckdb_for_drop)
wrapper.update_named_table_with_schema()
wrapper.update_with_local_file_paths()

In [43]:
duckdb_drop_res = wrapper.execute()

In [44]:
df_duckdb_substrait_drop = duckdb_drop_res.to_df()
df_duckdb_substrait_drop.head()

,VendorID,passenger_count,trip_distance,fare_per_distance_unit,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare,extra,mta_tax,tip_amount,tolls_amount,total_amount,congestion_surcharge,airport_fee
0,1,1.0,8.40,3.035714,1.0,N,79,116,1,25.5,3.0,0.5,5.85,0.0,35.15,2.5,0.0
1,1,1.0,0.90,5.555555,1.0,N,75,236,2,5.0,3.0,0.5,0.00,0.0,8.80,2.5,0.0
2,1,1.0,3.40,3.382353,1.0,N,236,168,2,11.5,3.0,0.5,0.00,0.0,15.30,2.5,0.0
3,1,1.0,0.00,NaN,1.0,N,47,61,1,44.2,0.0,0.5,0.00,0.0,45.00,0.0,0.0
4,2,1.0,1.96,4.591837,1.0,N,238,152,1,9.0,0.5,0.5,3.09,0.0,13.39,0.0,0.0


In [45]:
df_sql = spark.sql(query_drop)
spark_drop_df = df_sql.toPandas()
spark_drop_df.head()

,VendorID,passenger_count,trip_distance,fare_per_distance_unit,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare,extra,mta_tax,tip_amount,tolls_amount,total_amount,congestion_surcharge,airport_fee
0,1,1.0,8.40,3.035714,1.0,N,79,116,1,25.5,3.0,0.5,5.85,0.0,35.15,2.5,0.0
1,1,1.0,0.90,5.555555,1.0,N,75,236,2,5.0,3.0,0.5,0.00,0.0,8.80,2.5,0.0
2,1,1.0,3.40,3.382353,1.0,N,236,168,2,11.5,3.0,0.5,0.00,0.0,15.30,2.5,0.0
3,1,1.0,0.00,NaN,1.0,N,47,61,1,44.2,0.0,0.5,0.00,0.0,45.00,0.0,0.0
4,2,1.0,1.96,4.591837,1.0,N,238,152,1,9.0,0.5,0.5,3.09,0.0,13.39,0.0,0.0


In [46]:
assert spark_drop_df.equals(df_duckdb_substrait_drop)

We have done rename, add column, update column and drop column operations. Ad you can see the schema evolution is supported via Substrait for Duckdb. 

Similarly this can be used to support Schema evolution for other Substrait consumers.

## References

1. Tabular Iceberg: https://tabular.io/blog/introducing-tabular/
2. Tabular test suite for Iceberg, Spark: https://github.com/tabular-io/docker-spark-iceberg
3. Apache Iceberg: https://iceberg.apache.org/
4. Substrait: https://substrait.io/
5. Duckdb/Substrait: https://duckdb.org/docs/extensions/substrait.html
6. Ibis/Substrait: https://github.com/ibis-project/ibis-substrait
7. Experimental Module for PyIceberg and Python-Substrait: https://github.com/vibhatha/pyiceberg_substrait

## Credits

[Tabular](https://tabular.io) resources for Apache Iceberg have been very helpful to learn more about building tools around the existing eco-system. 